In [141]:
import openai
import os
from openai import OpenAI
import time
import json
from dotenv import load_dotenv
load_dotenv()

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai.api_key)
model = "gpt-3.5-turbo-1106"

# Assistants API with Code Interpreter
Following docs here: https://platform.openai.com/docs/assistants/tools

In [3]:
# Create an assistant with Code Interpreter
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview"
)

In [4]:
# Create a thread
thread = client.beta.threads.create()

In [5]:
# Create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [6]:
# List messages
thread_messages = client.beta.threads.messages.list(thread.id)
print("\nRaw thread_messages.data\n", thread_messages.data)

for idx, message in enumerate(reversed(thread_messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)


Raw thread_messages.data
 [ThreadMessage(id='msg_DZOrnzBAmW8trIpFr0UU0JJm', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1701459489, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_1iatFbxyMn9KT1eiFaxAWEHr')]

Message 0:
 I need to solve the equation `3x + 11 = 14`. Can you help me?


In [7]:
# Run the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account." # These instructions override the default instructions of the Assistant
)

# Check the run status. Expected output: in progress OR completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

Run status:  queued


In [13]:
# Check the run status again. Expected output: completed, move on to next step once completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

Run status:  in_progress


In [12]:
# Check the messages
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print("\nRaw messages:\n", messages)

for idx, message in enumerate(reversed(messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)


Raw messages:
 SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_YvXcPnfdGuVlajsxtYbtmYTO', assistant_id='asst_j29opJ5wU1SkkO1qFYqIGMIB', content=[MessageContentText(text=Text(annotations=[], value="Certainly, Jane Doe! To solve the equation `3x + 11 = 14`, we need to find the value of `x` that makes the equation true. We do this by isolating `x` on one side of the equation. Here are the steps:\n\n1. Subtract 11 from both sides of the equation to get `3x` by itself:\n    `3x + 11 - 11 = 14 - 11`\n2. This simplifies to:\n    `3x = 3`\n3. Finally, divide both sides by 3 to solve for `x`:\n    `x = 3 / 3`\n\nLet's calculate the exact value of `x`."), type='text')], created_at=1701459490, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_YEl7BLR38ZXUh4dyWzHuLvdx', thread_id='thread_1iatFbxyMn9KT1eiFaxAWEHr'), ThreadMessage(id='msg_DZOrnzBAmW8trIpFr0UU0JJm', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solv

# Assistants API with Retrieval

In [97]:
file = client.files.create(
  file=open("test.txt", "rb"),
  purpose='assistants'
)

In [98]:
# Create an assistant with Retrieval
assistant = client.beta.assistants.create(
  name="Reader",
  description="You are great at reading files. You read the .txt files present. You also share a brief text summary of what you observed.",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter", "type": "retrieval"}],
  file_ids=[file.id]
)

In [99]:
# Create a thread
thread = client.beta.threads.create()

In [100]:
# Create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Give me some summaries of the existing files"
)

In [101]:
# List messages
thread_messages = client.beta.threads.messages.list(thread.id)
print("\nRaw thread_messages.data\n", thread_messages.data)

for idx, message in enumerate(reversed(thread_messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)


Raw thread_messages.data
 [ThreadMessage(id='msg_nRHMZPdtiY8FGYXpVLJOb9HK', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Give me some summaries of the existing files'), type='text')], created_at=1701461122, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_HOVIrYhEYGdRPEvDgxI5HEoi')]

Message 0:
 Give me some summaries of the existing files


In [102]:
# Run the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Check the run status. Expected output: in progress OR completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

Run status:  queued


In [107]:
# Check the run status again. Expected output: completed, move on to next step once completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

# Check the messages
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print("\nRaw messages:\n", messages)

for idx, message in enumerate(reversed(messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)

assert(run.status == "completed")

Run status:  completed

Raw messages:
 SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_mwZfgZuHCZTpWvw6iMRfQwDE', assistant_id='asst_71fOZbXrxUUV9CfBbBc4GMk7', content=[MessageContentText(text=Text(annotations=[], value='The file you uploaded is named `test.txt` and contains a brief message. Here’s a summary:\n\n- The message thanks you for using Assistants API.\n- It mentions that this is indeed just a test text file.\n- It includes a personal fun fact stating that the author likes grapes.\n- There\'s a reference to a "special number," which is 100.\n\nThis appears to be a simple test file rather than a substantive document. If you have any specific requests or need more detailed summaries of other files, please let me know!'), type='text')], created_at=1701461129, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_MotJ4RtxdGcuXHtqJwZxNkuO', thread_id='thread_HOVIrYhEYGdRPEvDgxI5HEoi'), ThreadMessage(id='msg_nRHMZPdtiY8FGYXpVLJOb9HK', assistant_

In [108]:
# Playing around with using an assistant file
file_2 = client.files.create(
  file=open("test_2.txt", "rb"),
  purpose='assistants'
)

In [109]:
# Create the assistant file
assistant_file = client.beta.assistants.files.create(
  assistant_id=assistant.id, 
  file_id=file_2.id
)
print(assistant_file)

AssistantFile(id='file-FVDKn6HPii4fjEaURQK8sK1y', assistant_id='asst_71fOZbXrxUUV9CfBbBc4GMk7', created_at=1701461235, object='assistant.file')


In [110]:
# Retrieve the assistant file
assistant_file = client.beta.assistants.files.retrieve(
  assistant_id=assistant.id, 
  file_id=file_2.id
)
print(assistant_file)

AssistantFile(id='file-FVDKn6HPii4fjEaURQK8sK1y', assistant_id='asst_71fOZbXrxUUV9CfBbBc4GMk7', created_at=1701461235, object='assistant.file')


In [112]:
# List assistant files
assistant_files = client.beta.assistants.files.list(
  assistant_id=assistant.id
)
print("# of files: ", len(assistant_files.data))
print(assistant_files)

# of files:  2
SyncCursorPage[AssistantFile](data=[AssistantFile(id='file-FVDKn6HPii4fjEaURQK8sK1y', assistant_id='asst_71fOZbXrxUUV9CfBbBc4GMk7', created_at=1701461235, object='assistant.file'), AssistantFile(id='file-AkZfAfpPn09KBzw7eJk1rIqE', assistant_id='asst_71fOZbXrxUUV9CfBbBc4GMk7', created_at=1701461118, object='assistant.file')], object='list', first_id='file-FVDKn6HPii4fjEaURQK8sK1y', last_id='file-AkZfAfpPn09KBzw7eJk1rIqE', has_more=False)


In [114]:
# Upload a file with an "assistants" purpose
file_3 = client.files.create(
  file=open("test_3.txt", "rb"),
  purpose='assistants'
)

In [123]:
# Run the Assistants API trying to retrieve the higher number

# Create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id, # Continue the existing thread
    role="user",
    content="Tell me what the highest special number is based on the information in my files. I just added a new file.",
    file_ids=[file_3.id]
)

# List messages
thread_messages = client.beta.threads.messages.list(thread.id)
print("\nRaw thread_messages.data\n", thread_messages.data)

for idx, message in enumerate(reversed(thread_messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)


Raw thread_messages.data
 [ThreadMessage(id='msg_mNidzFPMtt1oCfI6q7T4opmY', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Tell me what the highest special number is based on the information in my files. I just added a new file.'), type='text')], created_at=1701461839, file_ids=['file-l0qW89pGfXmNubnhg5UTzkfD'], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_HOVIrYhEYGdRPEvDgxI5HEoi'), ThreadMessage(id='msg_Yi4kPHkZ4z9SUmgY71MYXZGo', assistant_id='asst_71fOZbXrxUUV9CfBbBc4GMk7', content=[MessageContentText(text=Text(annotations=[], value='Based on the information from the second file, `test_2.txt`, the special number mentioned is 10. Comparing this with the special number 100 from the first file, `test.txt`, the highest special number mentioned in the files provided is 100.'), type='text')], created_at=1701461738, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_fbS9Op9L7aJtSQMYF36Idsn6

In [134]:
# Run the assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# Check the run status. Expected output: in progress OR completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

Run status:  queued


In [135]:
# Check the run status again. Expected output: completed, move on to next step once completed.
# Also added annotations here. # TODO: Current example doesn't use annotations, so improve the sample by increasing the size of the text files to support annotations
while run.status == "in_progress" or run.status == "queued":
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )
  print("Run status: ", run.status)
  time.sleep(1)

# Check the messages
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print("\nRaw messages:\n", messages)

for idx, message in enumerate(reversed(messages.data)):
    # Adding annotations here

    # Extract the message content
    message_content = message.content[0].text
    annotations = message_content.annotations
    citations = []

    # Iterate over the annotations and add footnotes
    for index, annotation in enumerate(annotations):
        # Replace the text with a footnote
        message_content.value = message_content.value.replace(annotation.text, f' [{index}]')

        # Gather citations based on annotation attributes
        if (file_citation := getattr(annotation, 'file_citation', None)):
            cited_file = client.files.retrieve(file_citation.file_id)
            citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
        elif (file_path := getattr(annotation, 'file_path', None)):
            cited_file = client.files.retrieve(file_path.file_id)
            citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
            # Note: File download functionality not implemented above for brevity

    # Add footnotes to the end of the message before displaying to user
    message_content.value += '\n' + '\n'.join(citations)
    
    print(f"\nMessage {idx}:\n", message_content)

assert(run.status == "completed")

Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  in_progress
Run status:  completed

Raw messages:
 SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_KJN2J23bhjn4fEBZdjNBCg7w', assistant_id='asst_71fOZbXrxUUV9CfBbBc4GMk7', content=[MessageContentText(text=Text(annotations=[], value='I\'ve reviewed the content of the most recently uploaded file `test_3.txt` and found that it lists the "special number" as 250. Considering the previous files where the special numbers mentioned were 100 and 10, the highest special number based on the information in your files is 250.'), type='text')], created_at=1701462757, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_SpS0lzWgdFMJhTMJSmKUUn05', thread_id='thread_HOVIrYhEYGdRPEvDgxI5HEoi'), ThreadMessage(id='msg_JoD40xBF3PK4Ie7q7oXfja55', assistant_id='asst_71fOZbXrxUUV9CfBbBc4GMk7', content=[MessageCo

In [130]:
# Delete an assistant file
deleted_assistant_file = client.beta.assistants.files.delete(
    assistant_id=assistant.id,
    file_id=file_2.id
)
print(deleted_assistant_file)

FileDeleteResponse(id='file-FVDKn6HPii4fjEaURQK8sK1y', deleted=True, object='assistant.file.deleted')


In [131]:
# List assistant files
assistant_files = client.beta.assistants.files.list(
  assistant_id=assistant.id
)
print("# of files: ", len(assistant_files.data))
print(assistant_files)

# of files:  1
SyncCursorPage[AssistantFile](data=[AssistantFile(id='file-AkZfAfpPn09KBzw7eJk1rIqE', assistant_id='asst_71fOZbXrxUUV9CfBbBc4GMk7', created_at=1701461118, object='assistant.file')], object='list', first_id='file-AkZfAfpPn09KBzw7eJk1rIqE', last_id='file-AkZfAfpPn09KBzw7eJk1rIqE', has_more=False)


In [ ]:
# Cancelling a run. TODO: Integrate into sample somewhere to demonstrate.
# run = client.beta.threads.runs.cancel(thread_id=thread.id, run_id=run.id)
# while run.status == "in_progress":
#     run = client.beta.threads.runs.retrieve(
#         thread_id=thread.id,
#         run_id=run.id
#     )

# Assistants API with function calling

In [148]:
assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4-1106-preview",
  tools=[{
      "type": "function",
    "function": {
      "name": "getCurrentWeather()",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  }, {
    "type": "function",
    "function": {
      "name": "getNickname",
      "description": "Get the nickname of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    } 
  }]
)

In [149]:
# Create a thread
thread = client.beta.threads.create()

# Create a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's the weather for San Francisco?",
)

# Create a run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [152]:
# Check the run status. Expected output: in progress OR completed
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print("Run status: ", run.status)

Run status:  requires_action


In [151]:
# Functions hard coded for brevity
def getCurrentWeather(location, **kwargs):
    return "It's sunny!"

def getNickname(location):
    return "The city by the bay"

tools = {
    'getCurrentWeather': getCurrentWeather,
    'getNickname': getNickname
}

In [153]:
assert(run.status == "requires_action") # Needs to require action before calling tools

tool_outputs = []
for tool_call in run.required_action.submit_tool_outputs.tool_calls:
    tool_id, tool_function, tool_type = tool_call.id, tool_call.function, tool_call.type
    args = json.loads(tool_function.arguments)
    tool_output = tools[tool_function.name](**args)
    tool_outputs.append({
        "tool_call_id": tool_id,
        "output": tool_output
    })
print("tool_outputs\n", tool_outputs)

tool_outputs
 [{'tool_call_id': 'call_Y6QDy6t6DScEQfzM2TkaRpjp', 'output': "It's sunny!"}]


In [154]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=tool_outputs
)

In [155]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

print("\nRaw messages:\n", messages)

for idx, message in enumerate(reversed(messages.data)):
    print(f"\nMessage {idx}:\n", message.content[0].text.value)


Raw messages:
 SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_vMl6UwajaL41lc9OBWIn7RNp', assistant_id='asst_zyYnFhBP5E8mJrFuV8UeCRLB', content=[MessageContentText(text=Text(annotations=[], value='The current weather in San Francisco, CA is sunny! Enjoy the beautiful weather!'), type='text')], created_at=1701465127, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_8HeCKimlyLPeKa7wp4RAmtal', thread_id='thread_b0SnRaXwqT0UPNaGkw3aMske'), ThreadMessage(id='msg_3NjR5i9xIHDa7AM4Sqd6SBfy', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value="What's the weather for San Francisco?"), type='text')], created_at=1701465114, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_b0SnRaXwqT0UPNaGkw3aMske')], object='list', first_id='msg_vMl6UwajaL41lc9OBWIn7RNp', last_id='msg_3NjR5i9xIHDa7AM4Sqd6SBfy', has_more=False)

Message 0:
 What's the weather for San Francisco?

Message 1:
 The curre

In [ ]:
# TODO: Add unit tests and ensure that they're comprehensive to ensure that the API shape between AOAI and OAI are the same. Likely use seed if supported.